In [1]:
#Dependencies
import pandas as pd
from splinter import Browser
from bs4 import BeautifulSoup
import pymongo

import requests
import time

In [2]:
#Splinter
executable_path =  {'executable_path': 'chromedriver.exe'} 
browser = Browser('chrome', **executable_path, headless = False)

In [3]:
#NASA website news
news_url = "https://mars.nasa.gov/news/"
browser.visit(news_url)

html = browser.html
soup = BeautifulSoup(html, "html.parser")

#Get the most recent news in the page
news_title = soup.find(class_="content_title").text
news_content = soup.find(class_="article_teaser_body").text

print(news_title)
print("-----------------------------------------------------------------")
print(news_content)

NASA's Curiosity Rover Aims to Get Its Rhythm Back
-----------------------
Rover engineers at JPL will try to restore percussive drilling on Mars this week, part of a larger series of tests that will last through summer.


In [10]:
jpl_url = "https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars"
browser.visit(jpl_url)

browser.click_link_by_partial_text("FULL IMAGE")
time.sleep(10)
browser.click_link_by_partial_text("more info")
time.sleep(10)
browser.find_by_tag('figure').click()

html = browser.html
soup = BeautifulSoup(html, "html.parser")

featured_image_url  = soup.img['src']

print(featured_image_url)

https://www.jpl.nasa.gov/spaceimages/images/largesize/PIA19113_hires.jpg
